# AI Final Project: multi-class KNN fake news identifier

*Team Members: Josh Yiznitsky, Olivia Fountain, Luke Kaplan*

This notebook contains code for designing a multi-class KNN model for determining fake news.

In [3]:
#import data and add headers
import pandas as pd

def read_data(file):
    data = pd.read_csv(file, 
                            delimiter='\t',
                            header=None,
                            names=['id','label','statement','subject','speaker','job_title','state_info','party_affiliation','barely_true_counts','false_counts','half_true_counts','mostly_true_counts','pants_on_fire_counts','context'])
    return data

train = read_data('liar_dataset/train.tsv')
test = read_data('liar_dataset/test.tsv')
validate = read_data('liar_dataset/valid.tsv')


print("training data: ", train['label'].value_counts())
print("test data: ", test['label'].value_counts())
print("validation data: ", validate['label'].value_counts())

training data:  label
half-true      2114
false          1995
mostly-true    1962
true           1676
barely-true    1654
pants-fire      839
Name: count, dtype: int64
test data:  label
half-true      265
false          249
mostly-true    241
barely-true    212
true           208
pants-fire      92
Name: count, dtype: int64
validation data:  label
false          263
mostly-true    251
half-true      248
barely-true    237
true           169
pants-fire     116
Name: count, dtype: int64


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import pandas as pd

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_dataset(dataset):

    def preprocess_text(text):
        if not isinstance(text, str):
            return text
        # Lowercasing
        text = text.lower()
        # Remove non-alphanumeric characters
        text = re.sub(r'\W', ' ', text)
        # Remove all single characters
        text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
        # Substituting multiple spaces with single space
        text = re.sub(r'\s+', ' ', text, flags=re.I)

        # Tokenization
        tokens = text.split()

        # Removal of Stop Words and Lemmatization
        lem = WordNetLemmatizer()
        tokens = [lem.lemmatize(word) for word in tokens if word not in set(stopwords.words('english'))]
        return ' '.join(tokens)

    #drop all columns except label and statement
    dataset = dataset[['label', 'statement']]

    #remove rows with missing values
    dataset = dataset.dropna()

    #apply preprocessing to statement column
    dataset['statement'] = dataset['statement'].apply(preprocess_text)

    return dataset

#preprocess data training and test data
train = preprocess_dataset(train)
test = preprocess_dataset(test)
validate = preprocess_dataset(validate)

#create multi-class label set
multi_train = train.copy()
multi_test = test.copy()
multi_validate = validate.copy()

#add a column next to label with the number of the label
multi_train['label_num'] = multi_train['label'].map({'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5})
multi_test['label_num'] = multi_test['label'].map({'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5})
multi_validate['label_num'] = multi_validate['label'].map({'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5})

#create binary label set
binary_train = train.copy()
binary_test = test.copy()
binary_validate = validate.copy()

#convert labels to true and false for binary label set
binary_train['label'] = binary_train['label'].map({'pants-fire': 'false', 'false': 'false', 'barely-true': 'false', 'half-true': 'true', 'mostly-true': 'true', 'true': 'true'})
binary_test['label'] = binary_test['label'].map({'pants-fire': 'false', 'false': 'false', 'barely-true': 'false', 'half-true': 'true', 'mostly-true': 'true', 'true': 'true'})
binary_validate['label'] = binary_validate['label'].map({'pants-fire': 'false', 'false': 'false', 'barely-true': 'false', 'half-true': 'true', 'mostly-true': 'true', 'true': 'true'})


#add a column next to label with the number of the label
binary_train['label_num'] = binary_train['label'].map({'false': 0, 'true': 1})
binary_test['label_num'] = binary_test['label'].map({'false': 0, 'true': 1})
binary_validate['label_num'] = binary_validate['label'].map({'false': 0, 'true': 1})







[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joshyiz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/joshyiz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
print("multi_class training data: ")
multi_train.head(5)

multi_class training data: 


,label,statement,label_num
0,false,say annies list political group support third ...,1
1,half-true,decline coal start started natural gas took st...,3
2,mostly-true,hillary clinton agrees john mccain voting give...,4
3,false,health care reform legislation likely mandate ...,1
4,half-true,economic turnaround started end term,3


In [6]:
print("multi_class test data: ")
multi_test.head(5)

multi_class test data: 


,label,statement,label_num
0,true,building wall mexico border take literally year,5
1,false,wisconsin pace double number layoff year,1
2,false,say john mccain done nothing help vet,1
3,half-true,suzanne bonamici support plan cut choice medic...,3
4,pants-fire,asked reporter whether he center criminal sche...,0


In [7]:
print("multi_class validation data: ")
multi_validate.head(5)

multi_class validation data: 


,label,statement,label_num
0,barely-true,le american working 70,2
1,pants-fire,obama sworn office use holy bible instead kura...,0
2,false,say organization parading social welfare organ...,1
3,half-true,say nearly half oregon child poor,3
4,half-true,attack republican various program economic sti...,3


In [8]:
print("binary training data: ")
binary_train.head(5)

binary training data: 


,label,statement,label_num
0,false,say annies list political group support third ...,0
1,true,decline coal start started natural gas took st...,1
2,true,hillary clinton agrees john mccain voting give...,1
3,false,health care reform legislation likely mandate ...,0
4,true,economic turnaround started end term,1


In [9]:
print("binary test data: ")
binary_test.head(5)

binary test data: 


,label,statement,label_num
0,true,building wall mexico border take literally year,1
1,false,wisconsin pace double number layoff year,0
2,false,say john mccain done nothing help vet,0
3,true,suzanne bonamici support plan cut choice medic...,1
4,false,asked reporter whether he center criminal sche...,0


In [10]:
print("binary validation data: ")
binary_validate.head(5)

binary validation data: 


,label,statement,label_num
0,false,le american working 70,0
1,false,obama sworn office use holy bible instead kura...,0
2,false,say organization parading social welfare organ...,0
3,true,say nearly half oregon child poor,1
4,true,attack republican various program economic sti...,1


In [11]:
import spacy

nlp = spacy.load("en_core_web_lg")


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

# Create a TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# binary classification data
Xb_train_tfidf = tfidf_vectorizer.fit_transform(binary_train['statement'])
Xb_test_tfidf = tfidf_vectorizer.transform(binary_test['statement'])
Xb_validate_tfidf = tfidf_vectorizer.transform(binary_validate['statement'])

Yb_train = binary_train['label_num']
Yb_test = binary_test['label_num']
Yb_validate = binary_validate['label_num']

# multi-class classification data
Xm_train_tfidf = tfidf_vectorizer.fit_transform(multi_train['statement'])
Xm_test_tfidf = tfidf_vectorizer.transform(multi_test['statement'])
Xm_validate_tfidf = tfidf_vectorizer.transform(multi_validate['statement'])

Ym_train = multi_train['label_num']
Ym_test = multi_test['label_num']
Ym_validate = multi_validate['label_num']

print(Ym_test)

0       5
1       1
2       1
3       3
4       0
       ..
1262    3
1263    2
1264    2
1265    2
1266    1
Name: label_num, Length: 1267, dtype: int64


In [13]:
# create a KNN classifierfor binary classification
BinaryKnn = KNeighborsClassifier(n_neighbors=100, metric='euclidean')
BinaryKnn.fit(Xb_train_tfidf, Yb_train)

# Predict the labels
Yb_pred = BinaryKnn.predict(Xb_test_tfidf)

# Print the classification report
print("Binary Classification Report")
print(classification_report(Yb_test, Yb_pred))



Binary Classification Report
              precision    recall  f1-score   support

           0       0.63      0.32      0.42       553
           1       0.62      0.86      0.72       714

    accuracy                           0.62      1267
   macro avg       0.63      0.59      0.57      1267
weighted avg       0.63      0.62      0.59      1267



In [34]:
#print multi-class data
print(Ym_test)

print(Ym_train)





0       5
1       1
2       1
3       3
4       0
       ..
1262    3
1263    2
1264    2
1265    2
1266    1
Name: label_num, Length: 1267, dtype: int64
0        1
1        3
2        4
3        1
4        3
        ..
10235    4
10236    4
10237    3
10238    1
10239    0
Name: label_num, Length: 10240, dtype: int64


In [35]:

# create a KNN classifier for multi-class classification
MultiKnn = KNeighborsClassifier(n_neighbors=2, metric='euclidean')
MultiKnn.fit(Xm_train_tfidf, Ym_train)

# Predict the labels
Ym_pred = MultiKnn.predict(Xm_test_tfidf)

# Print the classification report
print("Multi-Class Classification Report")
print(classification_report(Ym_test, Ym_pred))


Multi-Class Classification Report
              precision    recall  f1-score   support

           0       0.16      0.17      0.17        92
           1       0.23      0.26      0.24       249
           2       0.21      0.18      0.20       212
           3       0.23      0.24      0.24       265
           4       0.25      0.44      0.32       241
           5       0.33      0.00      0.01       208

    accuracy                           0.23      1267
   macro avg       0.24      0.22      0.19      1267
weighted avg       0.24      0.23      0.20      1267



In [16]:
# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(Xb_train_tfidf, Yb_train)
best_rf_model = grid_search.best_estimator_

# Predict and evaluate
binary_predictions = best_rf_model.predict(Xb_test_tfidf)
print(classification_report(binary_test['label_num'], binary_predictions))

              precision    recall  f1-score   support

           0       0.65      0.17      0.27       553
           1       0.59      0.93      0.72       714

    accuracy                           0.60      1267
   macro avg       0.62      0.55      0.50      1267
weighted avg       0.62      0.60      0.52      1267



In [17]:
def add_vectors(dataset):
    dataset['vector'] = dataset['statement'].apply(lambda text: nlp(text).vector)
    return dataset

multi_train = add_vectors(multi_train)
multi_test = add_vectors(multi_test)
binary_train = add_vectors(binary_train)
binary_test = add_vectors(binary_test)



In [18]:
multi_train.head()

,label,statement,label_num,vector
0,false,say annies list political group support third ...,1,"[-0.09125201, -0.2517279, -0.13689697, 1.06777..."
1,half-true,decline coal start started natural gas took st...,3,"[-0.7686892, 0.5762692, -2.3380945, 0.43318465..."
2,mostly-true,hillary clinton agrees john mccain voting give...,4,"[-0.42389503, 0.18668796, -1.7479826, 0.480140..."
3,false,health care reform legislation likely mandate ...,1,"[-0.25652453, 1.6648048, -2.437448, 0.03223201..."
4,half-true,economic turnaround started end term,3,"[-2.139714, 1.5611031, -0.645526, 1.4543259, 3..."


In [19]:
multi_test.head()

,label,statement,label_num,vector
0,true,building wall mexico border take literally year,5,"[-0.30884856, -0.40162143, -2.9716258, 0.24982..."
1,false,wisconsin pace double number layoff year,1,"[-0.65460676, 1.1971833, 0.6555855, 1.644395, ..."
2,false,say john mccain done nothing help vet,1,"[1.9692186, 0.9003071, -2.135753, -0.63334715,..."
3,half-true,suzanne bonamici support plan cut choice medic...,3,"[-1.3098322, 1.0344467, -2.7497778, 0.15368445..."
4,pants-fire,asked reporter whether he center criminal sche...,0,"[-0.55129397, 0.8915434, -1.4848228, 0.9362866..."


In [20]:
binary_train.head()

,label,statement,label_num,vector
0,false,say annies list political group support third ...,0,"[-0.09125201, -0.2517279, -0.13689697, 1.06777..."
1,true,decline coal start started natural gas took st...,1,"[-0.7686892, 0.5762692, -2.3380945, 0.43318465..."
2,true,hillary clinton agrees john mccain voting give...,1,"[-0.42389503, 0.18668796, -1.7479826, 0.480140..."
3,false,health care reform legislation likely mandate ...,0,"[-0.25652453, 1.6648048, -2.437448, 0.03223201..."
4,true,economic turnaround started end term,1,"[-2.139714, 1.5611031, -0.645526, 1.4543259, 3..."


In [21]:
binary_test.head()

,label,statement,label_num,vector
0,true,building wall mexico border take literally year,1,"[-0.30884856, -0.40162143, -2.9716258, 0.24982..."
1,false,wisconsin pace double number layoff year,0,"[-0.65460676, 1.1971833, 0.6555855, 1.644395, ..."
2,false,say john mccain done nothing help vet,0,"[1.9692186, 0.9003071, -2.135753, -0.63334715,..."
3,true,suzanne bonamici support plan cut choice medic...,1,"[-1.3098322, 1.0344467, -2.7497778, 0.15368445..."
4,false,asked reporter whether he center criminal sche...,0,"[-0.55129397, 0.8915434, -1.4848228, 0.9362866..."


In [22]:
Xm_train = multi_train['vector'].to_list()
Xm_test = multi_test['vector'].to_list()
Yb_train = multi_train['label_num']
Ym_test = multi_test['label_num']

In [23]:


# Create KNN classifier for multi-class
MultiKnn = KNeighborsClassifier(n_neighbors = 3, metric='euclidean')

# Fit the classifier to the data
MultiKnn.fit(Xm_train, Ym_train)

# get the predictions
Multipredictions = MultiKnn.predict(Xm_test)

In [24]:
from sklearn.metrics import classification_report

print(classification_report(Ym_test, Multipredictions))

              precision    recall  f1-score   support

           0       0.12      0.26      0.16        92
           1       0.20      0.33      0.25       249
           2       0.16      0.19      0.18       212
           3       0.21      0.15      0.18       265
           4       0.20      0.11      0.14       241
           5       0.27      0.10      0.14       208

    accuracy                           0.19      1267
   macro avg       0.19      0.19      0.18      1267
weighted avg       0.20      0.19      0.18      1267



In [25]:
Xb_train = binary_train['vector'].to_list()
Xb_test = binary_test['vector'].to_list()
Yb_train = binary_train['label_num']
Ym_test = binary_test['label_num']

In [26]:
#Create KNN classifier for binary class
BinaryKnn = KNeighborsClassifier(n_neighbors = 7, metric='euclidean')

# Fit the classifier to the data
BinaryKnn.fit(Xb_train, Yb_train)

# get predictions
BinaryPredictions = BinaryKnn.predict(Xb_test)

In [27]:
print(classification_report(Ym_test, BinaryPredictions))

              precision    recall  f1-score   support

           0       0.51      0.52      0.51       553
           1       0.62      0.61      0.62       714

    accuracy                           0.57      1267
   macro avg       0.56      0.57      0.57      1267
weighted avg       0.57      0.57      0.57      1267

